In [ ]:
!pip install -q biopython scipy matplotlib seaborn


In [ ]:
import torch
from src.data.casp16_loader import CASP16DataLoader
from src.benchmarks.casp16_benchmark import CASP16Benchmark
from src.advanced_model import AdvancedProteinFoldingModel
from src.protein_embeddings import ESM2Embedder


In [ ]:
embedder = ESM2Embedder(model_name='esm2_t33_650M_UR50D')
model_quantum = AdvancedProteinFoldingModel(input_dim=1280, use_quantum=True)
model_classical = AdvancedProteinFoldingModel(input_dim=1280, use_quantum=False)


In [ ]:
loader = CASP16DataLoader(cache_dir='./data/casp16')
targets = loader.download_targets(categories=['Regular', 'FM'], force_refresh=False)
print(f'Loaded {len(targets)} CASP16 targets')


In [ ]:
benchmark = CASP16Benchmark(model_quantum, model_classical, embedder, device='cpu')
results = benchmark.run_full_benchmark(n_targets=min(50, len(targets)))


In [ ]:
from pathlib import Path
output_dir = Path('./casp16_results')
output_dir.mkdir(exist_ok=True)
benchmark.generate_casp16_report(results, output_dir)


In [ ]:
print('Statistical Comparison:')
print(f"Quantum Mean TM-score: {results['aggregate_stats']['quantum_mean_tm']:.3f}")
print(f"Classical Mean TM-score: {results['aggregate_stats']['classical_mean_tm']:.3f}")
print(f"p-value (Wilcoxon): {results['statistical_tests'].get('wilcoxon_pvalue', 1.0):.4f}")
print(f"Effect size (Cohen\'s d): {results['statistical_tests'].get('cohens_d', 0.0):.3f}")
